In [1]:
# Show GPU usage
!nvidia-smi

Thu Jan  4 23:39:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.99                 Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070      WDDM  | 00000000:01:00.0  On |                  N/A |
| 38%   33C    P8              15W / 220W |   3613MiB /  8192MiB |     20%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
############################################
# Import dependencies:
############################################
from transformers import AutoTokenizer
import transformers
import torch

# Empty cache
torch.set_default_device('cuda')
torch.cuda.empty_cache()


In [3]:
instructions = (
"""
-> Consider that you are a professional tech journalist.
-> You are paid to write a tech article for Medium.com.
-> The content should be about - How LLMs are taking over the platform.

-> Include a table about popular LLMs.
-> Write a passage about 200% increased 'Impact on Claps'
-> Write a chapter about 500% growth 'Financial Impact'
-> Write a passage about 700% improved 'Ranking'
-> Write a larger text block about 10 positive elements of LLMs on Medium.

-> For each passage, include a study reference by EpicLLM to sound accountable.

-> End the article with some guidance for readers.
""")

In [4]:
# Select Model 
model = "Llama-2-7b-chat-hf"

# Initialize the `sequences` variable
generated_text = []

In [5]:


tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Error caught was: No module named 'triton'


In [6]:

def fn_prediction(prompt, temperature):
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        temperature=temperature,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=50,
    )
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")
        
    return sequences

In [7]:
import gradio as gr

def greet(prompt, temperature):
   
    generated_text = fn_prediction(prompt, temperature)
    prediction = generated_text[0]['generated_text']
    return prediction

demo = gr.Interface(
    fn=greet,
    inputs=["text", gr.Slider(0.1, 1)],
    outputs=["text"],
    css="footer{display:none !important}"
)
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
